In [1]:
import json
import io
import bz2
import collections as coll
import nltk
from nltk.corpus import cmudict
import string
import math
import numpy as np
import pandas as pd
from string import maketrans
import regex as re
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('ks772', 'x27zf4qmhv')
import itertools as itr
import glob 
from nltk.stem.snowball import SnowballStemmer

SYLLABLE_DICT = cmudict.dict()  

In [3]:
# lower
def lower_list(word_list):
    new_list = []
    for word in word_list:
        new_list.append(word.lower())
    return new_list

In [4]:
## this only takes a string...
def remove_punct(text):
    #pat = re.compile(r"[\p{P}\p{S}]+")
    pat = re.compile(r"[\p{P}\p{S}](?<!-)+") #in python 3 its r not ur: (ur"[\p{P}\p{S}]+")
    return pat.sub("", text)

# strip punc
def strip_list(word_list):
    new_list = []
    for word in word_list:
        #word_np = remove_punct_word(word)
        new_list.append(word.strip())
    return new_list


In [5]:
#stem
def stem_remover(word_list):
    return word_list
#     new_list = []
#     stemmer = SnowballStemmer("english")
#     for word in word_list:
#         new_word = stemmer.stem(word)
#         new_list.append(new_word)
#     return new_list

In [6]:
# clean
def clean_list(word_list):
    new = lower_list(word_list)
    new_new = strip_list(word_list)
    return new_new
    

In [7]:
#is three syllbs
def is_three_plus_syllables(word):
    x = SYLLABLE_DICT.get(word.lower())
    if x is None:
        return False
    return len([y for y in x[0] if y[-1].isdigit()]) >= 3

In [8]:
#is poly sylb
def count_poly_syllables(sentence):
    words = sentence.split()
    return sum(is_three_plus_syllables(word) for word in words)



In [9]:
# count sentences
def count_sents(sentence):
    return len([''.join(y) for v,y in itr.groupby(sentence, lambda x: x not in '.!?') if v])

In [10]:
#compute smog
def compute_smog_score(list_tweets):
    grade = 0
    tot_num_pollys = 0
    total_num_sentences = 0

    for tweet in list_tweets:
        total_num_sentences += count_sents(tweet)
        tot_num_pollys += count_poly_syllables(tweet)

    if total_num_sentences <30: 
        print "too few tweets"
        return None

    else:
        grade = 1.0430*(math.sqrt(tot_num_pollys*(30.0/total_num_sentences))) + 3.1291
        return grade

In [11]:
#create sat list
def create_sat_list():
    sat_word_list = []
    with open('/Users/sports195admin/Desktop/rxch/github/research/SATGRE/SATWords.txt', 'r') as infile:
        for line in infile:
            sat_word_list.append(line.strip("\n").split()[0]) # add first word of every line to list
        clean_sat = clean_list(sat_word_list)
        return clean_sat

In [12]:
#create gre list
def create_gre_list():
    gre_word_list = []
    with open('/Users/sports195admin/Desktop/rxch/github/research/SATGRE/GREwords.txt', 'r') as infile:
        for line in infile:
            gre_word_list.append(line.strip("\n").split()[0]) # add first word of every line to list
        clean_gre = clean_list(gre_word_list)
        return clean_gre

In [49]:
# word match counter
# word_set_dict - dictionary with two keys ("sat" and "gre') - "sat" - sat word set

def word_match_counter(clean_words, word_set_dict):
    result = {k:0 for k in word_set_dict}
    total_count = len(clean_words)
    word_counter = coll.Counter(clean_words)
    for word, count in word_counter.items():
        for k, v in word_set_dict.items():
            if word in v:
                print(word)
                result[k] += count
    #print(result)            
    return {k:v/float(total_count) for k,v in result.items()}

In [58]:
# open files/texts/tweets

def main():
    print("loading words")
    sat_words = set(create_sat_list())
    gre_words = set(create_gre_list())

    dict_of_word_sets = {"sat":sat_words, "gre" : gre_words}
    print("done loading word list")

    in_file_name = "/Users/sports195admin/Downloads/success_timelines_00000.json.bz2"
    out_file_name = "/Users/sports195admin/Downloads/user_scores_00000.json.bz2"

    with bz2.BZ2File(in_file_name) as infile, bz2.BZ2File(out_file_name, 'w') as outfile:
        list_of_sat_nums = []   
        list_of_sat_percents = []
        list_of_gre_nums = []
        list_of_gre_percents = []

        print("beginning loading in text")
        for line in infile: 

            first_tab = line.index("\t") # location of first tab char
            first_tab
            id = line[:first_tab]

            timeline = json.loads(line[first_tab:]) # creates a dict

            user_id = timeline[u'user'][0]["id_str"] # gets user id as string

            list_tweets = [x['text'] for x in timeline[u'tweets']] # makes all the user's tweets into a list      
            print("got tweet list")
            smog_score = compute_smog_score(list_tweets) #computes the user's smog score and stores it in this var
            print("finished smog score")
            usable_sample = ""
        
            for tweet in list_tweets:
                usable_sample += remove_punct(tweet.lower()) # removes all punct and makes evrything lower
            print("merged clean tweets")
            list_words = usable_sample.split()
            
            stripped_list_words = strip_list(list_words)
            
            clean_words = stem_remover(stripped_list_words)
            
            word_count_scores = word_match_counter(clean_words, dict_of_word_sets)
            
            tdict = coll.OrderedDict() 
            tdict["uid"] = user_id
            tdict["smog"] = smog_score
            tdict["sat"] = word_count_scores["sat"]
            tdict["gre"] = word_count_scores["gre"]
            outfile.write(json.dumps(tdict)+'\n')
            print(tdict)
            
        

In [59]:
main()

loading words
done loading word list
beginning loading in text
got tweet list
finished smog score
merged clean tweets
explode
service
likely
episode
came
liquor
confident
cant
cant
head
check
roll
finally
minute
guy
aye
productive
course
album
butt
august
crush
despicable
expect
guess
aver
aver
believe
ready
lying
later
craving
guinea
off
quiet
OrderedDict([('uid', u'1012513417'), ('smog', 5.637255490524155), ('sat', 0.009841992501339047), ('gre', 0.003012854847348688)])
got tweet list
finished smog score
merged clean tweets
reign
ultimate
free
calorie
sarcasm
matter
came
bomb
cant
cant
roll
finally
guy
sacrifice
chastity
recover
later
ought
peaceful
check
generally
nowadays
sense
auburn
expect
alcohol
guess
off
believe
compliment
confidence
slack
liking
OrderedDict([('uid', u'1024997802'), ('smog', 6.21433560688645), ('sat', 0.014972067039106146), ('gre', 0.004469273743016759)])
got tweet list
finished smog score
merged clean tweets
involve
lax
facial
free
service
et
anatomy
check
epi

In [52]:
# X clean in sat and gre word list maker
# X open text tweet file(s)


        
# 1 compute smog

# 2 clean tweet text

# 3 match sat words and gre words using word match counter function somehow 




# fix strip func

#  make data table func
#  make histogram func


In [18]:
# lowerX
# stripX
# stemX
# cleanX
#     lower
#     strip
#     stem
# isthreesyllbsX
# countpollyX
# countsyllbsX
# count sentsX
# compute smog scoreX

# create sat list
#     clean
# create gre list
#     clean
# open files 
#     compute smog score
#     clean
# word match counter
#dt
#histogram

In [ ]:
print("loading words")
sat_words = set(create_sat_list())
gre_words = set(create_gre_list())

dict_of_word_sets = {"sat":sat_words, "gre" : gre_words}
print("done loading word list")

with open("/Users/sports195admin/Desktop/nytarticle.txt") as infile:
    y = infile.readlines()

user_id = 0
    
list_tweets = y # makes all the user's tweets into a list      
print("got tweet list")
smog_score = compute_smog_score(list_tweets) #computes the user's smog score and stores it in this var
print("finished smog score")
usable_sample = ""

for tweet in list_tweets[:10]:
    usable_sample += remove_punct(tweet.lower()) # removes all punct and makes evrything lower
print("merged clean tweets")
list_words = usable_sample.split()

stripped_list_words = strip_list(list_words)

clean_words = stem_remover(stripped_list_words)

word_count_scores = word_match_counter(clean_words, dict_of_word_sets)

tdict = coll.OrderedDict() 
tdict["uid"] = user_id
tdict["smog"] = smog_score
tdict["sat"] = word_count_scores["sat"]
tdict["gre"] = word_count_scores["gre"]

#outfile.write(json.dumps(tdict)+'\n')

print(tdict)


In [ ]:
print len(sat_words.intersection(gre_words))
print len(sat_words)
print len(gre_words)

In [ ]:
/Users/sports195admin/Desktop/ppr.txt

In [ ]:
print("loading words")
sat_words = set(create_sat_list())
gre_words = set(create_gre_list())
gre_not_sat = gre_words - sat_words

dict_of_word_sets = {"sat":sat_words, "gre" : gre_words, "gre only" : gre_not_sat}
print("done loading word list")

with open("/Users/sports195admin/Desktop/ppr.txt") as infile:
    y = infile.readlines()

user_id = 0
    
list_tweets = y # makes all the user's tweets into a list      
print("got tweet list")
smog_score = compute_smog_score(list_tweets) #computes the user's smog score and stores it in this var
print("finished smog score")
usable_sample = ""

for tweet in list_tweets[:10]:
    usable_sample += remove_punct(tweet.lower()) # removes all punct and makes evrything lower
print("merged clean tweets")
list_words = usable_sample.split()

stripped_list_words = strip_list(list_words)

clean_words = stem_remover(stripped_list_words)

word_count_scores = word_match_counter(clean_words, dict_of_word_sets)

tdict = coll.OrderedDict() 
tdict["uid"] = user_id
tdict["smog"] = smog_score
tdict["sat"] = word_count_scores["sat"]
tdict["gre"] = word_count_scores["gre"]
tdict["gre only"] = word_count_scores["gre only"]

#outfile.write(json.dumps(tdict)+'\n')

print(tdict)

